In [1]:
!pip install -q sequence
!pip install -q matplotlib
!pip install -q ipywidgets

In [2]:
from enum import Enum, auto

class MsgType(Enum):
    PING = auto()
    PONG = auto()

In [3]:
from sequence.topology.node import Node
from sequence.protocol import Protocol
from sequence.message import Message

class PingProtocol(Protocol):
    def __init__(self, owner: Node, name: str, other_name: str, other_node: str):
        super().__init__(owner, name)
        owner.protocols.append(self)
        self.other_name = other_name
        self.other_node = other_node

    def init(self):
        pass

    def start(self):
        new_msg = Message(MsgType.PING, self.other_name)
        self.owner.send_message(self.other_node, new_msg)

    def received_message(self, src: str, message: Message):
        assert message.msg_type == MsgType.PONG
        print("node {} received pong message at time {}".format(self.owner.name, self.owner.timeline.now()))


class PongProtocol(Protocol):
    def __init__(self, owner: Node, name: str, other_name: str, other_node: str):
        super().__init__(owner, name)
        owner.protocols.append(self)
        self.other_name = other_name
        self.other_node = other_node

    def init(self):
        pass

    def received_message(self, src: str, message: Message):
        assert message.msg_type == MsgType.PING
        print("node {} received ping message at time {}".format(self.owner.name, self.owner.timeline.now()))
        new_msg = Message(MsgType.PONG, self.other_name)
        self.owner.send_message(self.other_node, new_msg)

/usr/local/lib/python3.10/dist-packages/qutip/__init__.py:66: UserWarning: The new version of Cython, (>= 3.0.0) is not supported.
  warnings.warn(


In [4]:
from sequence.kernel.timeline import Timeline
from sequence.components.optical_channel import ClassicalChannel

tl = Timeline(1e12)

node1 = Node("node1", tl)
node2 = Node("node2", tl)

cc0 = ClassicalChannel("cc0", tl, 1e3, 1e9)
cc1 = ClassicalChannel("cc1", tl, 1e3, 1e9)
cc0.set_ends(node1, node2.name)
cc1.set_ends(node2, node1.name)

pingp = PingProtocol(node1, "pingp", "pongp", "node2")
pongp = PongProtocol(node2, "pongp", "pingp", "node1")

In [5]:
from sequence.kernel.process import Process
from sequence.kernel.event import Event

process = Process(pingp, "start", [])
event = Event(0, process)
tl.schedule(event)

tl.init()
tl.run()

node node2 received ping message at time 1000000000
node node1 received pong message at time 2000000000
